# Описание ДЗ1.

На основе семинара 1 предложите 2 метода улучшения построения эмбеддингов вопросов на основе word vectors.

За задание можно получить максимум 10 баллов (за каждый метод можно получить максимум 5 баллов).

Разбалловка:
*   **Воспроизводимость и читабельность кода - 1 балл** (все воспроизвелось и все понятно для проверяющего - 1 балл; непонятный код и/или ничего не воспроизвелось - 0 баллов).
*   **Корректность метода - 1 балл** (метод математически корректен и с точки зрения логики кода нет ошибок - 1 балл; все остальные случаи - 0 баллов).
*   **Описание метода в техническом отчете - 2 балла** (есть подробное описание метода и почему используете именно его с обоснованиями - 2 балла; есть описание метода, но нет обоснования - 1 балл; нет описания метода- 0 баллов).
*   **Иновационность - 1 балл** (используете нетривиальную обработку/лосс-функцию/модель - 1 балл; просто перебираете модели из коробок - 0 баллов).

!!! ДЗ необходимо выполнять только в Google Colab !!!

Присылать на почту llmrisks@yandex.ru с номером ДЗ и ФИО в теме. Каждая ДЗ отдельным письмом с отдельной темой.

# 1. Информация о сабмите

**Мартин Михаил Алексеевич**

# 2. Технический отчет

Методы улучшения построения эмбеддингов фраз на основе эмбеддингов входящих слов, использованные в данном домашнем задании:
1. Обработка входного текста  
   Мотивация: Так как размер обучающей выборки мал, хотелось ужать словарь токенов, удалив шумные токены и объеденив очень похожие.

   - Приводим все символы в словах к строчной форме (объединение схожих)  
   - Удаляем [стоп-слова](https://en.wikipedia.org/wiki/Stop_word) (удалаяем шумные)  
   - Удаляем знаки препинания и прочие символы (удаляем шумные)  
   - [Лемматизируем](https://ru.wikipedia.org/wiki/Лемматизация) токены, т.е. приводим их к нормальной форме (объеденение схожих)

2. Обработка эмбеддингов входящих слов  
   Мотивация: Когда мы усредняем эмбеддинги, обычно считается, что каждый токен имеет равный вес, но на самом деле это не всегда справедливо. Лучше использовать взвешенное среднее, где ключевым словам присваивается больший вес. Так можно точнее передать значимость ключевых слов в эмбеддинге всего предложения.  
   Для этого можно воспользоваться методом [TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF), делая акцент на весе IDF, поскольку частота появления слов (TF) уже учитывается при суммировании эмбеддингов.


$$
\text{IDF}(w, D) = \log\left(\frac{|D|}{|{d∈D:w∈d|}}\right)
$$

# 3. *Code*

## 3.1 Импорт библиотек

In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import WordPunctTokenizer

import gensim.downloader as api
import numpy as np
from scipy.spatial.distance import cosine
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
from pymystem3 import Mystem

In [2]:
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## 3.2 Загрузка данных

In [3]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

data = open("./quora.txt", encoding="utf-8").read().split("\n")

--2024-12-13 14:40:22--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/p0t2dw6oqs6oxpd6zz534/quora.txt?rlkey=bjupppwua4zmd4elz8octecy9&dl=1 [following]
--2024-12-13 14:40:23--  https://www.dropbox.com/scl/fi/p0t2dw6oqs6oxpd6zz534/quora.txt?rlkey=bjupppwua4zmd4elz8octecy9&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc783b2d262895f7c97e29e875fe.dl.dropboxusercontent.com/cd/0/inline/CgJiZ7w1gAAqUJCnG_OixDJFqODy6gsMTG5-Jkf0u8ZKZgYi1yOAQeZckj8j9hwx249HWjCb8bAqAlGttnhDMwrxnapc6g6YpNJl0SB_mkux65QfCnoA1MrRIGY6gcdeLE8/file?dl=1# [following]
--2024-12-13 14:40:23--  https://uc783b2d262895f7c97e29e875fe.dl.dropboxusercontent.com/cd/0/inline/CgJiZ7w1gAA

## 3.3. Модель

In [4]:
model = api.load("glove-twitter-100")

In [5]:
class QA_Helper:

    def __init__(self, tokenizer, stop_words, lemmer, model, tfidf_vectorizer) -> None:
        self.tokenizer = tokenizer
        self.stop_words = stop_words
        self.lemmer = lemmer
        self.model = model
        self.tfidf_vectorizer = tfidf_vectorizer
        self.train_data = None

    def fit(self, data: list[str]) -> None:

        # перед расчётом TF-IDF предобрабатываем данные
        data = self.preprocessed(data)
        # обучаем TF-IDF модуль
        self.tfidf_vectorizer.fit(data)
        # сохраняем IDF
        self.idfs = {
            word: self.tfidf_vectorizer.idf_[i]
            for i, word in enumerate(self.tfidf_vectorizer.get_feature_names_out())
        }
        # находим эмбеддинги для обучающих данных
        self.train_data = self.get_phrase_embeddings(data)

    def get_phrase_embeddings(self, phrase: list[str] | str) -> list[np.ndarray] | np.ndarray:

        if isinstance(phrase, list):
            return list(map(self.get_phrase_embeddings, phrase))
        else:
            phrase = self.preprocessed(phrase)

        # снова разбиваем на токены
        phrase = self.tokenizer.tokenize(phrase)

        # получаем эмбеддинги слов в фразе
        words_embeddings = [self.model.get_vector(token) for token in phrase]
        # преобразуем их в numpy-массив
        words_embeddings = np.array(words_embeddings)

        # подтягиваем IDF'ы для токенов в фразе
        idfs = [self.idfs.get(token, 1) for token in phrase]
        # преобразуем их в numpy-массив
        idfs = np.array(idfs)[:, np.newaxis]

        # получаем эмбеддинги фразы, как средневзвешенное эмбеддингов слов в фразе на IDF
        if words_embeddings.size > 0:
            phrase_embeddings = words_embeddings * idfs
            phrase_embeddings = words_embeddings.mean(axis=0)
        else:
            phrase_embeddings = np.zeros([self.model.vector_size], dtype="float32")

        return phrase_embeddings

    def preprocessed(self, phrase: list[str] | str) -> list[str] | str:

        if isinstance(phrase, list):
            return list(map(self.preprocessed, phrase))

        # приводим к нижнему регистру
        phrase = phrase.lower()
        # разбиваем на токены
        phrase = self.tokenizer.tokenize(phrase)
        # отбрасываем стоп-слова
        phrase = [token for token in phrase if token not in self.stop_words]
        # отбрасываем знаки пунктуации и прочие символы
        phrase = [token for token in phrase if token.isalpha()]
        # лематизируем токены
        phrase = self.lemmer.lemmatize(" ".join(phrase))
        # отбрасывем токены, которые не знает модель
        phrase = self.tokenizer.tokenize(phrase)
        phrase = [token for token in phrase if token in self.model.key_to_index]
        phrase = " ".join(phrase)

        return phrase

    def find_nearest(self, query, k: int = 10) -> list[str]:
        """
        Для данного вопроса возвращает k наиболее похожих на него вопросов.

        Возвращаемые вопросы упорядочены по убыванию схожести.
        Под схожестью понимается косинусное расстояние между векторами-эмбеддингами.

        """
        # получаем эмбеддинг данного вопроса
        embedding_query = self.get_phrase_embeddings(query)
        # находим вектор схожести с обучающими данными
        similarities = np.array([
            1 - cosine(embedding_query, embedding_data)
            for embedding_data in tqdm(self.train_data)
        ])
        # вытаскиваем топ-k схожих вопросов
        indices = np.argsort(similarities)[::-1]
        indices = np.argpartition(-similarities, range(10))[:k]

        return [data[i] for i in indices]

## 3.4 Тестирование

In [6]:
qa_helper = QA_Helper(
    tokenizer=WordPunctTokenizer(),
    stop_words=stopwords.words("english"),
    lemmer=WordNetLemmatizer(),
    model=model,
    tfidf_vectorizer=TfidfVectorizer(),
)
qa_helper.fit(data)

In [7]:
qa_helper.find_nearest("How do i enter the matrix?")

  0%|          | 0/537273 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/scipy/spatial/distance.py:647: RuntimeWarning: invalid value encountered in divide
  dist = 1.0 - uv / math.sqrt(uu * vv)
100%|██████████| 537273/537273 [00:06<00:00, 81961.06it/s]


['How do I enter color codes in excel?',
 'Where do I enter my free ride code for Lyft?',
 'What should I do to enter hollywood?',
 'How can I enter into Hollywood?',
 'What is your review of Enter Contests to Win Prizes?',
 'What is the best platform to enter in quiz competitions and earn?',
 'Where can I get an ebook of A Quest of Heroes parts for free?',
 'Can I use PlayStation VR Launch Bundle in other countries?',
 'How did Disney enter India?',
 'How do you enter boot menu in Lenovo?']

In [8]:
qa_helper.find_nearest("How does Trump?")

100%|██████████| 537273/537273 [00:08<00:00, 64666.06it/s]


['What is Trump?',
 'What do we do now that Trump has won?',
 'How trump won?',
 'Donald trump won . What now?',
 'Who is Donald Trump?',
 'Is Donald Trump antifragile?',
 'Is Donald Trump an isolationist?',
 'Is Donald trump a populist?',
 'Will Donald Trump destroy Donald Trump?',
 'How did Melania Trump meet Donald Trump?']

In [9]:
qa_helper.find_nearest("Why don't i ask a question myself?")

100%|██████████| 537273/537273 [00:07<00:00, 76596.75it/s]


['Why do I ask this question?',
 'How do I ask a question on this?',
 'How do I ask a question?',
 'Can you ask me a question?',
 'How do you ask a question?',
 'Are there Quorans who ask a question and answer it themselves?',
 'Why do we ask questions?',
 'Why do we ask all these questions?',
 'What questions to ask any drdummer?',
 'What questions should we ask ourselves?']